<a href="https://colab.research.google.com/github/denis5ds/IA-sis420/blob/main/FinalIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SNAKE

In [ ]:
!pip install pygame



>Configurar el Entorno de Juego



In [ ]:
import numpy as np           # Biblioteca para manejar operaciones matemáticas y de arreglos (aunque no se usa en este código).
import random                # Biblioteca para generar números aleatorios.
import pygame                # Biblioteca para crear el entorno gráfico del juego.

# numero de estados 10 x 10=100 estados
WIDTH, HEIGHT = 200, 200  # Dimensiones de la ventana del juego (ancho y alto en píxeles).
BLOCK_SIZE = 20           # Tamaño de cada bloque de la cuadrícula (tamaño de un "paso" de la serpiente en píxeles).
GRID_WIDTH, GRID_HEIGHT = WIDTH // BLOCK_SIZE, HEIGHT // BLOCK_SIZE
# Número de bloques en la cuadrícula (ancho y alto) calculados dividiendo el tamaño total por el tamaño de cada bloque.

# Funciones auxiliares

def get_state(snake, food):
    """
    Define el estado actual del juego.
    - snake: lista de posiciones (bloques) que componen la serpiente.
    - food: posición de la comida en la cuadrícula.
    Devuelve una tupla que describe:
    1. Relación de posición entre la cabeza de la serpiente y la comida.
    2. Si hay peligro en las direcciones posibles (izquierda, derecha, arriba, abajo).
    """
    head = snake[0]  # Obtiene la posición de la cabeza de la serpiente.

    # Determina si hay peligro (si la serpiente chocaría consigo misma) en cada dirección.
    dir_l = head[0] > 0 and (head[0] - 1, head[1]) in snake  # Peligro a la izquierda.
    dir_r = head[0] < GRID_WIDTH - 1 and (head[0] + 1, head[1]) in snake  # Peligro a la derecha.
    dir_u = head[1] > 0 and (head[0], head[1] - 1) in snake  # Peligro arriba.
    dir_d = head[1] < GRID_HEIGHT - 1 and (head[0], head[1] + 1) in snake  # Peligro abajo.

    # Define el estado basándose en la posición de la comida y los peligros en cada dirección.
    state = (
        head[0] < food[0],  # Verdadero si la comida está a la derecha de la cabeza.
        head[0] > food[0],  # Verdadero si la comida está a la izquierda de la cabeza.
        head[1] < food[1],  # Verdadero si la comida está abajo de la cabeza.
        head[1] > food[1],  # Verdadero si la comida está arriba de la cabeza.
        dir_l,              # Verdadero si hay peligro a la izquierda.
        dir_r,              # Verdadero si hay peligro a la derecha.
        dir_u,              # Verdadero si hay peligro arriba.
        dir_d               # Verdadero si hay peligro abajo.
    )
    return state  # Devuelve el estado como una tupla booleana.

  def move_snake(snake, direction):
      """
      Mueve la serpiente en la dirección especificada.
      - snake: lista de posiciones (bloques) que componen la serpiente.
      - direction: entero que indica la dirección del movimiento:
          0 = Izquierda, 1 = Derecha, 2 = Arriba, 3 = Abajo.
      Devuelve la nueva lista de posiciones de la serpiente después de moverse.
      """
      head = snake[0]  # Obtiene la posición actual de la cabeza de la serpiente.
      if direction == 0:  # Movimiento hacia la izquierda.
          new_head = (head[0] - 1, head[1])
      elif direction == 1:  # Movimiento hacia la derecha.
          new_head = (head[0] + 1, head[1])
      elif direction == 2:  # Movimiento hacia arriba.
          new_head = (head[0], head[1] - 1)
      elif direction == 3:  # Movimiento hacia abajo.
          new_head = (head[0], head[1] + 1)
      # Actualiza la posición de la serpiente: la nueva cabeza se agrega al inicio y el último bloque se elimina.
      return [new_head] + snake[:-1]

def place_food(snake):
    """
    Genera una nueva posición para la comida en la cuadrícula.
    - snake: lista de posiciones (bloques) que componen la serpiente.
    La comida no puede aparecer en una posición ocupada por la serpiente.
    """
    while True:
        # Genera coordenadas aleatorias dentro de la cuadrícula.
        food = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))
        if food not in snake:  # Asegura que la comida no se coloque en el cuerpo de la serpiente.
            return food  # Devuelve la posición válida para la comida.

Configurar el Algoritmo Q-learning

In [ ]:
# Configuración del agente Q-learning
ALPHA = 0.1  # Tasa de aprendizaje: controla cuánto de la nueva información obtenida se incorpora a los valores Q existentes.
GAMMA = 0.9  # Factor de descuento: determina la importancia de las recompensas futuras en comparación con las inmediatas.
EPSILON = 1.0  # Probabilidad inicial de exploración: comienza alta para fomentar que el agente explore al principio.
EPSILON_DECAY = 0.995  # Factor de decaimiento de EPSILON: reduce gradualmente la probabilidad de exploración en cada episodio.
EPSILON_MIN = 0.01  # Valor mínimo de EPSILON: garantiza que siempre haya algo de exploración (aunque muy pequeña).

q_table = {}  # Diccionario que almacena los valores Q para cada combinación de estado y acción: clave=(estado, acción), valor=valor Q.

def get_q(state, action):
    return q_table.get((state, action), 0)  # Devuelve el valor Q para el par (estado, acción) si existe; si no, devuelve 0.

def choose_action(state):
    if random.random() < EPSILON:
        return random.randint(0, 3)  # Explorar: elige una acción al azar.
    else:
        return np.argmax([get_q(state, a) for a in range(4)])  # Explotar: elige la acción con el mayor valor Q para este estado.

Ejecutar el Entrenamiento

In [ ]:
# Inicializar pygame
pygame.init()  # Inicia los módulos de pygame.
screen = pygame.display.set_mode((WIDTH, HEIGHT))  # Crea la ventana del juego con las dimensiones especificadas.
clock = pygame.time.Clock()  # Inicializa el reloj para controlar la velocidad de actualización del juego.

for episode in range(200):  # Reduce el número de episodios
    snake = [(GRID_WIDTH // 2, GRID_HEIGHT // 2)]  # Posición inicial de la serpiente, centrada en la cuadrícula.
    food = place_food(snake)  # Genera la comida en una posición aleatoria.
    direction = 0  # Dirección inicial de la serpiente.
    score = 0  # Reinicia el puntaje del episodio.
    done = False  # Bandera para verificar si el episodio ha terminado.
    steps = 0  # Contador de pasos realizados por la serpiente en este episodio.

    while not done and steps < 500:  # Límite de pasos por episodio
        state = get_state(snake, food)  # Obtiene el estado actual del entorno.
        action = choose_action(state)  # Selecciona la acción usando la política ε-greedy.

        # Mover la serpiente
        new_snake = move_snake(snake, action)# Calcula la nueva posición de la serpiente.
        new_head = new_snake[0] # Obtiene la posición de la cabeza después de moverse.

        # Evaluar recompensa y estado
        if new_head == food:# La serpiente come la comida.
            new_snake.append(snake[-1])  # Crecer la serpiente
            food = place_food(new_snake) # Genera nueva comida.
            reward = 10 #Recompensa positiva por comer.
        elif (
            new_head in snake  # Colisión con el cuerpo
            or new_head[0] < 0  # Fuera del límite izquierdo
            or new_head[1] < 0 # Fuera del límite superior
            or new_head[0] >= GRID_WIDTH # Fuera del límite derecho
            or new_head[1] >= GRID_HEIGHT
        ):
            reward = -10  # Recompensa negativa por perder.
            done = True   # Termina el episodio.
        else:
            reward = -1   # Penalización por cada paso sin comer.

        # Actualizar Q-table
        next_state = get_state(new_snake, food)  # Calcula el próximo estado.
        best_next_action = max(get_q(next_state, a) for a in range(4))# Obtiene el mejor valor Q del próximo estado.
        q_table[(state, action)] = get_q(state, action) + ALPHA * (
            reward + GAMMA * best_next_action - get_q(state, action)
        )

        snake = new_snake  # Actualiza la posición de la serpiente.
        score += reward  # Suma la recompensa al puntaje total.
        steps += 1  # Incrementa el contador de pasos.

        # Reducir EPSILON para menos exploración
        EPSILON = max(EPSILON * EPSILON_DECAY, EPSILON_MIN)  # Reduce la probabilidad de exploración.

        # Mostrar el juego
        screen.fill((0, 0, 0))# Limpia la pantalla con color negro.
        for segment in snake:
            pygame.draw.rect(
                screen,
                (0, 255, 0),# Dibuja la serpiente en color verde.
                pygame.Rect(segment[0] * BLOCK_SIZE, segment[1] * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE),
            )
        pygame.draw.rect(
            screen,
            (255, 0, 0),# Dibuja la comida en color rojo.
            pygame.Rect(food[0] * BLOCK_SIZE, food[1] * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE),
        )
        pygame.display.flip() # Actualiza la pantalla.
        clock.tick(5)  # Controla la velocidad del juego (5 cuadros por segundo).

    print(f"Episode {episode + 1}: Score = {score}, Steps = {steps}")

    #imprime tabla q
    # Imprimir la tabla Q
print("\nTabla Q (Estado -> Acción -> Valor):")
for (state, action), value in q_table.items():
    print(f"Estado: {state}, Acción: {action}, Valor Q: {value:.2f}")


Episode 1: Score = -57, Steps = 48
Episode 2: Score = -59, Steps = 50
Episode 3: Score = -17, Steps = 30
Episode 4: Score = -500, Steps = 500
Episode 5: Score = -155, Steps = 157
Episode 6: Score = -10, Steps = 12
Episode 7: Score = -7, Steps = 9
Episode 8: Score = -500, Steps = 500
Episode 9: Score = -16, Steps = 18
Episode 10: Score = -73, Steps = 64
Episode 11: Score = -91, Steps = 82
Episode 12: Score = -38, Steps = 29
Episode 13: Score = -9, Steps = 11
Episode 14: Score = -500, Steps = 500
Episode 15: Score = -6, Steps = 8
Episode 16: Score = -334, Steps = 336
Episode 17: Score = -17, Steps = 19
Episode 18: Score = -42, Steps = 44
Episode 19: Score = -7, Steps = 9
Episode 20: Score = -1, Steps = 3
Episode 21: Score = -3, Steps = 5
Episode 22: Score = -5, Steps = 7
Episode 23: Score = -20, Steps = 22
Episode 24: Score = -6, Steps = 8
Episode 25: Score = -8, Steps = 10
Episode 26: Score = -5, Steps = 7
Episode 27: Score = -12, Steps = 14
Episode 28: Score = -4, Steps = 6
Episode 29: